In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# FDTD Library is open source MEEP
# https://meep.readthedocs.io/en/latest/

import meep as mp
#from meep.materials import SiO2, Si

%matplotlib widget

### Simulation Settings

In [ ]:

# To understand MEEP units see https://meep.readthedocs.io/en/latest/Introduction/#units-in-meep
wavelength = 1.55 # wavelength can have arbitrary length units (we choose um)
f = 1/wavelength # Frequency is defined in "natural units" as (1 um)/wavelength

l,w,z = [100,75,0] # length,width,height of the simulation region (um)
r = 30 # Star Coupler Radius (um)
h = 25 # Star Coupler Height (um) (this gives the flat top/bottom if desired)

waveguide_width = 0.5 # width of Si waveguide (um)
nguides_p = 10 # number of input waveguides above center (total numper is 2*nguides_p+1)
nangles = np.arange(-nguides_p,nguides_p+1,1) 

amplitudes = np.ones(len(nangles),dtype=complex) # amplitude/phase shift for each sources

## For tilted input
# amplitudes = np.zeros(len(nangles),dtype=complex) # amplitude/phase shift for each sources
# dl = 2*np.pi/len(nangles)
# for n in range(len(nangles)):
#     amplitudes[n]=np.exp(-1j*n)


y_guide_size = 1.5 # width of eigenmode source (needs to enclose waveguide)
dr = 10 # srces are placed at distance r+dr from confocal point

neff = 2.44 # Effective slab index from Lumerical simulation
#neff = 2.85

res = 30 # sim resolution (pixels/um)

### Create simulation region and define media

In [ ]:
# Define Simulation Cell
# All length units are microns
cell = mp.Vector3(l,w,z)

si = mp.Medium(epsilon=12)
sio2 = mp.Medium(index=1.444)

### Create Starcoupler Region as MP Material Grid

In [ ]:
x = np.arange(-r,r,1/res)
y = np.arange(-1.5*h/2,1.5*h/2,1/res)
xx,yy = np.meshgrid(x,y)

c1 = ((xx-r/2)**2 + yy**2 < r**2) # First circle with radius r
c2 = ((xx+r/2)**2 + yy**2 < r**2) # Second circle with radius r
b = (yy<h/2) & (yy>-h/2) # Rectangle allowing top/bottom cutoff

sc = c1 & c2 & b # star coupler shape comes from logical-AND all 3 shapes

star_grid = mp.MaterialGrid([len(x),len(y),0],
                           medium1 = sio2,
                           medium2 = si,
                           weights = sc.transpose())

star_block = mp.Block(size=mp.Vector3(x[-1]-x[0],y[-1]-y[0],0),material=star_grid)


### Create Input and Output Waveguides

In [ ]:
rot_angles = [np.arcsin(i*np.sqrt((1.55)/(r*len(nangles)*neff))) for i in nangles]

def make_waveguide(th,l,r):
    wg = mp.Block(mp.Vector3(np.abs(l),waveguide_width,mp.inf),
                     center=mp.Vector3(0,0),
                     e1=mp.Vector3(x=1).rotate(mp.Vector3(z=1), th),
                     e2=mp.Vector3(y=1).rotate(mp.Vector3(z=1), th),
                     material=si)
    
    wg = wg.shift(mp.Vector3(x=r/2-l*np.cos(th)/2,y=-l*np.sin(th)/2))
    return wg

input_waveguides = [make_waveguide(th,l,r) for th in rot_angles]
output_waveguides = [make_waveguide(th,-l,-r) for th in rot_angles]

### Add Eigenmode Sources

In [ ]:

def wg_eig_src(r,dr,th,ysize=1,freq=f,bd=1,amp=1.0):
    eig_src = mp.EigenModeSource(src=mp.ContinuousSource(frequency=freq),
                                 size=mp.Vector3(y=ysize),
                                 direction=mp.NO_DIRECTION,
                                 eig_kpoint=mp.Vector3(x=1).rotate(mp.Vector3(z=1),-th),
                                 center=mp.Vector3(-(r+dr)*np.cos(th)+r/2,(r+dr)*np.sin(th),0),
                                 eig_band = bd,
                                 eig_parity=mp.ODD_Z,
                                 eig_match_freq=True,
                                 amplitude=amp)
    return eig_src

# Adds an eigenmode source for each input waveguide -- can get a source's amplitude to 0 to turn it off
sources = [wg_eig_src(r,dr,s,ysize=y_guide_size,amp=a) for (s,a) in zip(rot_angles,amplitudes)]



### Create Simulation Object

In [ ]:
sim = mp.Simulation(resolution = res,
                    cell_size = cell,
                    default_material=sio2,
                    geometry = [star_block] + output_waveguides + input_waveguides,
                    sources=sources,
                    boundary_layers=[mp.PML(2.0)],
                    symmetries=[])

### Initialize Simulation

In [ ]:
# Separating the initialization call from the run call allows the option of viewing the simulation geometry/index before running the simulation
sim.init_sim()

### Run Simulation

In [ ]:
sim.run(until=300)

### Visualize E-field

In [ ]:
f1,ax1 = plt.subplots()

sim.plot2D(ax=ax1,output_plane=mp.Volume(center=mp.Vector3(), size=mp.Vector3(l,w,0)),
               fields=mp.Ez,
               field_parameters={'alpha':0.9})